202224111120
段志清
22科技一班

In [4]:
import cv2
import numpy as np

def get_frame(cap, scaling_factor):
    ret, frame = cap.read()
    if not ret:  # 如果读取失败
        return None
    frame = cv2.resize(frame, None, fx=scaling_factor, 
                      fy=scaling_factor, interpolation=cv2.INTER_AREA)
    return frame

if __name__ == '__main__':
    # 确保视频路径正确
    video_path = 'A_Warrior_Woman_Dressed_in_a_Medieval_Dress.mp4'
    cap = cv2.VideoCapture(video_path)
    
    if not cap.isOpened():
        print(f"无法打开视频文件: {video_path}")
        exit()
    
    scaling_factor = 0.5
    
    while True:
        frame = get_frame(cap, scaling_factor)
        if frame is None:  # 视频结束
            break
            
        # 转换为HSV色彩空间
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        
        # 定义绿色范围 (绿幕通常在这个范围)
        lower_green = np.array([35, 43, 46])
        upper_green = np.array([77, 255, 255])
        
        # 创建掩膜
        mask = cv2.inRange(hsv, lower_green, upper_green)
        mask_inv = cv2.bitwise_not(mask)
        
        # 应用掩膜
        foreground = cv2.bitwise_and(frame, frame, mask=mask_inv)
        
        # 平滑边缘
        kernel = np.ones((3,3), np.uint8)
        mask_inv = cv2.morphologyEx(mask_inv, cv2.MORPH_OPEN, kernel)
        mask_inv = cv2.morphologyEx(mask_inv, cv2.MORPH_CLOSE, kernel)
        
        # 创建透明背景
        bgra = cv2.cvtColor(frame, cv2.COLOR_BGR2BGRA)
        bgra[:,:,3] = mask_inv
        
        cv2.imshow('Original', frame)
        cv2.imshow('Foreground', foreground)
        cv2.imshow('Transparent BG', bgra)
        
        if cv2.waitKey(30) & 0xFF == 27:  # 按ESC退出
            break
            
    cap.release()
    cv2.destroyAllWindows()

In [6]:
import cv2
import numpy as np

def get_frame(cap, scaling_factor):
    ret, frame = cap.read()
    if not ret:  # 如果读取失败
        return None
    frame = cv2.resize(frame, None, fx=scaling_factor, 
                      fy=scaling_factor, interpolation=cv2.INTER_AREA)
    return frame

if __name__ == '__main__':
    cap = cv2.VideoCapture('A_Warrior_Woman_Dressed_in_a_Medieval_Dress.mp4')
    bg_subtractor = cv2.createBackgroundSubtractorMOG2()
    
    # 初始学习几帧作为背景
    for _ in range(10):
        frame = get_frame(cap, 0.5)
        if frame is None:
            break
        bg_subtractor.apply(frame, learningRate=1.0)
    
    # 重置视频
    cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
    
    while True:
        frame = get_frame(cap, 0.5)
        if frame is None:
            break
            
        # 应用背景减法
        fg_mask = bg_subtractor.apply(frame, learningRate=0)
        
        # 后处理
        fg_mask = cv2.morphologyEx(fg_mask, cv2.MORPH_OPEN, np.ones((3,3), np.uint8))
        fg_mask = cv2.morphologyEx(fg_mask, cv2.MORPH_CLOSE, np.ones((5,5), np.uint8))
        
        # 创建透明背景
        bgra = cv2.cvtColor(frame, cv2.COLOR_BGR2BGRA)
        bgra[:,:,3] = fg_mask
        
        cv2.imshow('Original', frame)
        cv2.imshow('Foreground Mask', fg_mask)
        cv2.imshow('Extracted Person', bgra)
        
        c = cv2.waitKey(30)
        if c == 27:
            break
            
    cap.release()
    cv2.destroyAllWindows()